### List of libaries, websites, and other resources referenced for this project:

Markdown Guide - Basic Syntax (for documentation): https://www.markdownguide.org/basic-syntax/ <br>

OpenAI's Gym library: https://github.com/openai/gym <br>
Gymnasium, a maintained fork of OpenAI's Gym library: https://github.com/Farama-Foundation/Gymnasium <br>

nes-py, an NES emulator and OpenAI Gym interface: https://github.com/Kautenja/nes-py <br>
gym-super-mario-bros, an OpenAI Gym environment for Super Mario Bros. using nes-py: https://github.com/Kautenja/gym-super-mario-bros <br>

PyTorch, a machine learning library for Python: https://pytorch.org/ <br>
Stable Baselines3, an RL algorithms library for PyTorch: https://github.com/DLR-RM/stable-baselines3 <br>

In [ ]:
# Install PyTorch (stable version) locally on Windows w/ GPU acceleration via pip
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install Stable Baselines3 library for our reinforcement learning algorithms (includes gymnasium)
%pip install stable-baselines3

### Note:

We are using WIP branches of nes-py & gym-super-mario-bros with gymnasium compatability fixes, hence the prebuilt wheels. <br>
nes-py: https://github.com/Kautenja/nes-py/pull/94 <br>
gym-super-mario-bros: https://github.com/Kautenja/gym-super-mario-bros/pull/127 <br>

In [ ]:
# Install prebuilt wheels for nes-py v8.3.0 & gym-super-mario-bros v7.4.0
%pip install ./external/nes_py-8.3.0-cp38-cp38-win_amd64.whl
%pip install ./external/gym_super_mario_bros-7.4.0-py3-none-any.whl

### Code

In [ ]:
# Import libraries
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

env = gym_super_mario_bros.make("SuperMarioBros-v0") # Make gym environment with SMB1 in standard ROM mode
env = JoypadSpace(env, SIMPLE_MOVEMENT) # Set joypad wrapper; restrict to SIMPLE_MOVEMENT (only 7 actions)

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()